In [ ]:
import FuturesClient

obj = FuturesClient.Manager()

symbol = "ADAUSDT"
side = "SELL"
quantity = 8
position_side = "BOTH"
limit_price = 0.8055
stop_price = 0.8050
time_in_force = "IOC"
reduce_only = False

order_signal = obj.take_profit_market_order(symbol=symbol,
                                           side=side,
                                           quantity=quantity,
                                           stop_price=stop_price)

In [3]:
order_signal

{'orderId': 50145649724,
 'symbol': 'ADAUSDT',
 'status': 'NEW',
 'clientOrderId': 'S3bniQFDuB70Wd2TJpc39x',
 'price': '0.80550',
 'avgPrice': '0.00',
 'origQty': '8',
 'executedQty': '0',
 'cumQty': '0',
 'cumQuote': '0.00000',
 'timeInForce': 'IOC',
 'type': 'TAKE_PROFIT',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'SELL',
 'positionSide': 'BOTH',
 'stopPrice': '0.80500',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'TAKE_PROFIT',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'EXPIRE_MAKER',
 'goodTillDate': 0,
 'updateTime': 1739323898160}